<a href="https://colab.research.google.com/github/aljunalei/rag-thesis-cloud/blob/main/LiPAD_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- STEP 1: FORCE ALIGN DEPENDENCIES ---
print("🔧 Aligning environment dependencies...")
!pip install -q \
    requests==2.32.4 \
    opentelemetry-api==1.37.0 \
    opentelemetry-sdk==1.37.0 \
    opentelemetry-proto==1.37.0 \
    opentelemetry-exporter-otlp-proto-common==1.37.0 \
    opentelemetry-exporter-otlp-proto-http==1.37.0 \
    opentelemetry-exporter-otlp-proto-grpc==1.37.0 \
    google-adk==1.17.0

# --- STEP 2: INSTALL YOUR RAG STACK ---
print("📚 Installing RAG libraries...")
!pip install -q \
    langchain langchain-core langchain-community langchain-huggingface \
    pypdf pymupdf sentence-transformers faiss-cpu chromadb \
    python-dotenv fastapi uvicorn[standard] pydantic[email] python-multipart \
    pyngrok bitsandbytes accelerate

print("✅ Setup Complete. Ignore any 'pip' red error text above if 'Successfully installed' appears.")

🔧 Aligning environment dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 6.1 MB/s eta 0:00:00
📚 Installing RAG libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 11.9 MB/s eta 0:00:00
  

In [ ]:
!pip install -q streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 131.5 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import os
import torch
import shutil
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

st.set_page_config(page_title="Thesis RAG", layout="wide")
DB_DIR = "./chroma_db"
DATA_DIR = "./data"

st.markdown("""
<style>
    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap');

    * {
        font-family: 'Inter', -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, sans-serif;
    }

    .main {
        background-color: #000000;
        padding: 0;
    }

    .stApp {
        background-color: #000000;
    }

    .block-container {
        padding-top: 0;
        padding-bottom: 120px;
        max-width: 680px;
    }

    [data-testid="stSidebar"] {
        background-color: #000000;
        border-right: 1px solid #2f3336;
    }

    [data-testid="stSidebar"] * {
        color: #e7e9ea !important;
    }

    .header-bar {
        position: sticky;
        top: 0;
        z-index: 1000;
        background-color: rgba(0, 0, 0, 0.65);
        backdrop-filter: blur(12px);
        border-bottom: 1px solid #2f3336;
        padding: 12px 16px;
        margin-bottom: 0;
    }

    .header-title {
        font-size: 20px;
        font-weight: 700;
        color: #e7e9ea;
        margin: 0;
        letter-spacing: -0.3px;
    }

    .tweet-card {
        background-color: #000000;
        border-bottom: 1px solid #2f3336;
        padding: 12px 16px;
        transition: background-color 0.2s ease;
        position: relative;
    }

    .tweet-card:hover {
        background-color: rgba(255, 255, 255, 0.03);
    }

    .tweet-main {
        display: flex;
        gap: 12px;
    }

    .tweet-avatar {
        width: 48px;
        height: 48px;
        border-radius: 50%;
        flex-shrink: 0;
        display: flex;
        align-items: center;
        justify-content: center;
        font-weight: 700;
        font-size: 20px;
        background: linear-gradient(135deg, #1d9bf0, #0c7abf);
        color: #ffffff;
    }

    .avatar-user {
        background: linear-gradient(135deg, #1d9bf0, #0c7abf);
    }

    .avatar-assistant {
        background: linear-gradient(135deg, #7856ff, #5b3fd1);
    }

    .tweet-body {
        flex: 1;
        min-width: 0;
    }

    .tweet-header {
        display: flex;
        align-items: center;
        gap: 4px;
        margin-bottom: 4px;
    }

    .tweet-name {
        font-weight: 700;
        font-size: 15px;
        color: #e7e9ea;
        line-height: 20px;
    }

    .tweet-handle {
        font-weight: 400;
        font-size: 15px;
        color: #71767b;
        line-height: 20px;
    }

    .tweet-content {
        font-size: 15px;
        line-height: 20px;
        color: #e7e9ea;
        word-wrap: break-word;
        white-space: pre-wrap;
        margin-top: 2px;
    }

    .reply-indicator {
        position: absolute;
        left: 39px;
        top: 68px;
        bottom: -12px;
        width: 2px;
        background-color: #2f3336;
    }

    .reply-card {
        border-left: 2px solid #2f3336;
        margin-left: 39px;
        padding-left: 28px;
    }

    .input-box {
        position: fixed;
        bottom: 0;
        left: 0;
        right: 0;
        background-color: rgba(0, 0, 0, 0.65);
        backdrop-filter: blur(12px);
        border-top: 1px solid #2f3336;
        padding: 12px 16px;
        z-index: 1000;
    }

    .stChatInput {
        max-width: 680px;
        margin: 0 auto;
    }

    .stChatInput > div {
        border-radius: 20px !important;
        border: 1px solid #2f3336 !important;
        background-color: #000000 !important;
    }

    .stChatInput input {
        color: #e7e9ea !important;
        font-size: 15px !important;
        background-color: transparent !important;
    }

    .stChatInput input::placeholder {
        color: #71767b !important;
    }

    h1, h2, h3, h4, h5, h6, p, span, div, label {
        color: #e7e9ea !important;
    }

    .stButton button {
        background-color: #1d9bf0;
        color: #ffffff;
        border-radius: 20px;
        border: none;
        font-weight: 700;
        padding: 8px 16px;
        transition: background-color 0.2s ease;
        font-size: 15px;
    }

    .stButton button:hover {
        background-color: #1a8cd8;
    }

    .stTextInput input {
        background-color: #000000;
        color: #e7e9ea;
        border: 1px solid #2f3336;
        border-radius: 4px;
    }

    .stSpinner > div {
        border-top-color: #1d9bf0 !important;
    }

    .stSuccess {
        background-color: rgba(0, 186, 124, 0.1);
        color: #00ba7c;
        border-radius: 8px;
    }

    .stInfo {
        background-color: rgba(29, 155, 240, 0.1);
        color: #1d9bf0;
        border-radius: 8px;
    }

    .stError {
        background-color: rgba(249, 24, 128, 0.1);
        color: #f91880;
        border-radius: 8px;
    }

    [data-testid="stChatInput"] {
        background-color: transparent;
    }

    .thinking-indicator {
        display: flex;
        align-items: center;
        gap: 8px;
        color: #71767b;
        font-size: 14px;
        padding: 12px 16px;
    }

    .dot-pulse {
        display: flex;
        gap: 4px;
    }

    .dot-pulse span {
        width: 8px;
        height: 8px;
        border-radius: 50%;
        background-color: #1d9bf0;
        animation: pulse 1.4s infinite ease-in-out;
    }

    .dot-pulse span:nth-child(1) {
        animation-delay: -0.32s;
    }

    .dot-pulse span:nth-child(2) {
        animation-delay: -0.16s;
    }

    @keyframes pulse {
        0%, 80%, 100% {
            opacity: 0.4;
            transform: scale(0.8);
        }
        40% {
            opacity: 1;
            transform: scale(1);
        }
    }
</style>
""", unsafe_allow_html=True)

@st.cache_resource
def load_llm(hf_token):
    model_id = "unsloth/gemma-2-9b-it-bnb-4bit"
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            device_map="auto",
            token=hf_token
        )
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=512,
            temperature=0.3,
            return_full_text=False
        )
        return HuggingFacePipeline(pipeline=pipe)
    except Exception as e:
        st.error(f"Model Error: {e}")
        return None

@st.cache_resource
def setup_vector_db():
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    if not os.path.exists(DB_DIR) or not os.listdir(DB_DIR):
        if not os.path.exists(DATA_DIR):
            return None, "⚠️ Error: 'data' folder not found. Please create it and upload files."
        with st.spinner("Processing Documents..."):
            pdf_loader = DirectoryLoader(DATA_DIR, glob="**/*.pdf", loader_cls=PyPDFLoader)
            pdf_docs = pdf_loader.load()
            txt_loader = DirectoryLoader(DATA_DIR, glob="**/*.txt", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
            txt_docs = txt_loader.load()
            documents = pdf_docs + txt_docs
            if not documents:
                return None, "⚠️ No documents found in 'data' folder."
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
            chunks = text_splitter.split_documents(documents)
            vectordb = Chroma.from_documents(
                documents=chunks,
                embedding=embeddings,
                persist_directory=DB_DIR
            )
            return vectordb, f"✅ Indexed {len(documents)} files ({len(chunks)} chunks)."
    else:
        vectordb = Chroma(persist_directory=DB_DIR, embedding_function=embeddings)
        return vectordb, "✅ Database Loaded."

with st.sidebar:
    st.title("⚙️ Settings")
    st.markdown("---")
    if 'HF_TOKEN' not in st.session_state:
        token = st.text_input("Hugging Face Token", type="password", placeholder="hf_...")
        if token:
            st.session_state['HF_TOKEN'] = token
            st.rerun()
    else:
        st.success("✓ Authenticated")
    st.info("📦 Model: Unsloth Gemma-2-9b")
    st.markdown("---")
    if st.button("🔄 Refresh Database"):
        if os.path.exists(DB_DIR):
            shutil.rmtree(DB_DIR)
        st.rerun()

st.markdown('<div class="header-bar"><div class="header-title">📘 Thesis RAG</div></div>', unsafe_allow_html=True)

if 'HF_TOKEN' in st.session_state:
    vectordb, status = setup_vector_db()
    if vectordb:
        llm = load_llm(st.session_state['HF_TOKEN'])
        if llm:
            st.toast(status)
            if "messages" not in st.session_state:
                st.session_state.messages = [{"role": "assistant", "content": "Hey! Ask me anything about your thesis or source code. I'm here to help! 🚀"}]

            for idx, msg in enumerate(st.session_state.messages):
                is_last_user = msg["role"] == "user" and idx < len(st.session_state.messages) - 1

                if msg["role"] == "user":
                    st.markdown(f"""
                    <div class="tweet-card">
                        <div class="tweet-main">
                            <div class="tweet-avatar avatar-user">Y</div>
                            <div class="tweet-body">
                                <div class="tweet-header">
                                    <span class="tweet-name">You</span>
                                    <span class="tweet-handle">@user</span>
                                </div>
                                <div class="tweet-content">{msg["content"]}</div>
                            </div>
                        </div>
                        {f'<div class="reply-indicator"></div>' if is_last_user else ''}
                    </div>
                    """, unsafe_allow_html=True)
                else:
                    is_reply = idx > 0 and st.session_state.messages[idx-1]["role"] == "user"
                    card_class = "tweet-card reply-card" if is_reply else "tweet-card"

                    st.markdown(f"""
                    <div class="{card_class}">
                        <div class="tweet-main">
                            <div class="tweet-avatar avatar-assistant">AI</div>
                            <div class="tweet-body">
                                <div class="tweet-header">
                                    <span class="tweet-name">Thesis Assistant</span>
                                    <span class="tweet-handle">@thesis_ai</span>
                                </div>
                                <div class="tweet-content">{msg["content"]}</div>
                            </div>
                        </div>
                    </div>
                    """, unsafe_allow_html=True)

            if prompt := st.chat_input("What's on your mind?"):
                st.session_state.messages.append({"role": "user", "content": prompt})

                with st.spinner(""):
                    retriever = vectordb.as_retriever(search_kwargs={"k": 4})
                    docs = retriever.invoke(prompt)
                    context_text = "\n\n".join([d.page_content for d in docs])
                    rag_prompt = f"Context:\n{context_text}\n\nQuestion: {prompt}\n\nAnswer:"
                    response = llm.invoke(rag_prompt)
                    st.session_state.messages.append({"role": "assistant", "content": response})

                st.rerun()
    else:
        st.error(status)
else:
    st.info("👈 Please enter your Hugging Face token in the sidebar to get started.")

Writing app.py


In [ ]:
import os
import time
import subprocess
from pyngrok import ngrok, conf
import getpass
import signal

# --- 1. FORCE INSTALL STREAMLIT ---
print("🔧 Ensuring Streamlit is installed...")
subprocess.run(["pip", "install", "-q", "streamlit", "pyngrok"])

# --- 2. AUTHENTICATE NGROK ---
if not conf.get_default().auth_token:
    print("Paste Ngrok Token:")
    conf.get_default().auth_token = getpass.getpass()

# --- 3. KILL OLD PROCESSES (AGGRESSIVE) ---
print("🔪 Clearing old sessions...")
ngrok.kill()

# Kill all streamlit processes
os.system("pkill -9 streamlit")
os.system("pkill -9 -f 'streamlit run'")

# Kill any process using port 8501
os.system("lsof -ti:8501 | xargs kill -9 2>/dev/null")

# Wait for cleanup
time.sleep(2)

# --- 4. START STREAMLIT ---
print("🚀 Starting Streamlit...")
log_file = open('streamlit_logs.txt', 'w')

process = subprocess.Popen(
    ['python', '-m', 'streamlit', 'run', 'app.py',
     '--server.port', '8501',
     '--server.address', '127.0.0.1',
     '--server.headless', 'true',
     '--browser.gatherUsageStats', 'false'],
    stdout=log_file,
    stderr=log_file
)

# --- 5. HEALTH CHECK ---
print("⏳ Initializing (Waiting 8 seconds)...")
time.sleep(8)

if process.poll() is not None:
    print("\n❌ ERROR: Streamlit crashed! Logs:")
    print("="*50)
    with open('streamlit_logs.txt', 'r') as f:
        print(f.read())
    print("="*50)
else:
    try:
        public_url = ngrok.connect(addr="127.0.0.1:8501", bind_tls=True).public_url
        print(f"\n✅ YOUR APP IS LIVE: {public_url}")
        print(f"📱 Access it from anywhere using the URL above")
        print(f"🔒 Connection is secure (HTTPS)")
        print(f"\n💡 Keep this terminal running to maintain the tunnel")
    except Exception as e:
        print(f"❌ Ngrok Error: {e}")
        print(f"💡 Try: 1) Check your ngrok token, 2) Restart ngrok service")

🔧 Ensuring Streamlit is installed...
Paste Ngrok Token:
··········
🔪 Clearing old sessions...
🚀 Starting Streamlit...
⏳ Initializing (Waiting 8 seconds)...

✅ YOUR APP IS LIVE: https://styliform-ephraim-unsignified.ngrok-free.dev
📱 Access it from anywhere using the URL above
🔒 Connection is secure (HTTPS)

💡 Keep this terminal running to maintain the tunnel


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
